# OMMX Message

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Jij-Inc/ommx/main?labpath=notebooks%2Fmessage.ipynb) 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jij-Inc/ommx/blob/main/notebooks/message.ipynb)

OMMX defines two main data formats:

1. **OMMX Message**: Represents mathematical programming problems/instances
2. **OMMX Artifact**: Represents other related artifacts like metadata.

In this tutorial notebook, we'll focus primarily usage of OMMX Message. The design concept of OMMX message is discussed in [the design document](https://github.com/Jij-Inc/ommx/blob/main/MESSAGE.md). You can learn more about OMMX Artifact in the [design document](https://github.com/Jij-Inc/ommx/blob/main/ARTIFACT.md) and [Notebook](https://github.com/Jij-Inc/ommx/blob/main/notebooks/artifact.ipynb)

In [1]:
# To run this notebook on Colab, please uncomment following line and run this cell
# ! pip install ommx ommx-python-mip-adapter

## Creating an OMMX Instance

An `Instance` is the fundamental OMMX object representing a mathematical programming problem. Let's see how to create one for a simple problem.

### Example: Knapsack Problem

Consider the following Knapsack problem:

$$
\begin{align*}
& \text{Maximize} & \sum_{i=1}^n p_i x_i & \\
&\text{Subject to} & \sum_{i=1}^n w_i x_i \leq W & ,\space x_i \in \{0, 1\} \quad (i = 1, \ldots, n)
\end{align*}
$$

Here:
- $p_i$ is the profit of item $i$ 
- $w_i$ is the weight of item $i$
- $W$ is the knapsack capacity
- $x_i$ are the binary decision variables (to be determined by solving the problem)

To construct the `Instance`, we'll follow these steps:

1. Define the decision variables
2. Build the objective function 
3. Specify the constraints
4. Create the `Instance`

Let's walk through each step:

In [2]:
from ommx.v1 import Instance, DecisionVariable

# Problem data
p = [10, 13, 18, 31, 7, 15]  # Profits
w = [11, 15, 20, 35, 10, 33] # Weights
W = 47  # Knapsack capacity

# Step 1: Define decision variables
# x[i] is 1 if item i is selected, 0 otherwise
x = [DecisionVariable.binary(i) for i in range(6)]

# Step 2: Build objective function
# Maximize total profit
objective = sum(p[i] * x[i] for i in range(6))

# Step 3: Specify constraints 
# Total weight must be <= knapsack capacity
constraint = sum(w[i] * x[i] for i in range(6)) <= W

# Step 4: Create the Instance
instance = Instance.from_components(
    decision_variables=x,
    objective=objective,
    constraints=[constraint],
    sense=Instance.MAXIMIZE,
)

### Decision Variables

In OMMX, each decision variable has a unique ID, specified when creating the `DecisionVariable` object. In the example above, the ID is simply the index `i` in the list comprehension.

The `DecisionVariable` class represents the variable's ID and kind (continuous, binary, integer):

In [3]:
x[1]

DecisionVariable(raw=id: 1
kind: KIND_BINARY
)

### Linear Functions

The objective and constraints are represented as linear functions of the decision variables.

Since the variable values are unknown until the problem is solved, OMMX uses a symbolic representation. A linear function like $2x_1 + 3x_2$ is encoded as a list of (variable ID, coefficient) pairs:

In [4]:
2 * x[1] + 3 * x[2]

Linear(raw=terms {
  id: 1
  coefficient: 2
}
terms {
  id: 2
  coefficient: 3
}
)

The `DecisionVariable` class overloads `+` and `*` to allow building linear expressions easily.

- For the objective, simply build the linear function to be maximized/minimized
- For constraints, use comparison operators `<=`, `>=`, `==` on linear expressions

In [5]:
3 * x[1] + 4 * x[3] <= 10

Constraint(raw=id: 1
equality: EQUALITY_LESS_THAN_OR_EQUAL_TO_ZERO
function {
  linear {
    terms {
      id: 1
      coefficient: 3
    }
    terms {
      id: 3
      coefficient: 4
    }
    constant: -10
  }
}
)

Constraints are automatically assigned unique IDs when created this way. You can also manually assign IDs - see the [Constraint API docs](https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/constraint_pb2/index.html#ommx.v1.constraint_pb2.Constraint) for details.

The `Instance` object collects all of this information - the variables, objective, and constraints - into a single problem specification.

In [6]:
byte_array = instance.to_bytes()

This `byte_array` is a binary representation of the `Instance` object based on the [OMMX Message schema](https://jij-inc.github.io/ommx/protobuf.html) defined by the [Protocol Buffers](https://protobuf.dev/). The `Instance` object can be deserialized from this binary representation by calling `Instance.from_bytes` method.

In [7]:
instance = Instance.from_bytes(byte_array)

The main advantage of Protocol Buffers is that it is language and platform independent. The `Instance` object can be serialized in Python and deserialized in other languages such as C++ or Rust and vice versa. This is useful when you want to create a problem in Python on your laptop, and solve it in C++ or Rust on a server. See [MESSAGE.md](https://github.com/Jij-Inc/ommx/blob/main/MESSAGE.md) for more details about entire design.

Note that OMMX is not designed as a modeler library, and modeler API is very limited. The above example is just a demonstration of what data is stored in `Instance` object for better understanding of serialized data.

## Solving 

OMMX itself doesn't include solvers. Instead, it provides a standard format for representing problems that can be consumed by various solvers via adapters.

Let's see how to solve our Knapsack problem using the Python-MIP solver via the `ommx-python-mip-adapter` package:

In [8]:
import ommx_python_mip_adapter as adapter

result = adapter.solve(instance)

The returned `result` is an `ommx.v1.Result` object containing the solution if the problem was solved successfully:

In [9]:
assert result.HasField("solution")  
solution = result.solution

The main part of the solution are the values of the decision variables $x_i$, stored in the `state` field:

In [10]:
print(solution.state.entries)

{3: 1.0, 2: 0.0, 5: 0.0, 1: 0.0, 0: 1.0, 4: 0.0}


Here $x[0] = x[3] = 1$ and the rest are $0$, giving a total profit of $p[0] + p[3] = 10 + 31 = 41$:

In [11]:
print(solution.objective)

41.0


OMMX normalizes constraints to the form $f(x) \leq 0$. For our knapsack constraint:

$$
f(x) = \sum_{i=1}^n w_i x_i - W \leq 0
$$

Plugging in the solution $x$ gives $f(x) = w[0] + w[3] - W = 11 + 35 - 47 = -1$, stored in the `evaluated_value` field:

In [12]:
assert len(solution.evaluated_constraints) == 1  
print(solution.evaluated_constraints[0].evaluated_value)

-1.0
